In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import random


In [2]:
# 设置随机数种子
def set_seed(seed):
    torch.manual_seed(seed)  # 设置PyTorch的随机数种子
    np.random.seed(seed)     # 设置Numpy的随机数种子
    random.seed(seed)       # 设置Python内置随机数生成器的种子
    # 如果使用CUDA
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # 如果使用多GPU
        # 以下两个设置可以帮助保持CUDA生成的随机数在每次运行时保持一致
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# 使用
seed = 42  # 设置随机数种子
set_seed(seed)

In [3]:
from models.Unet import Unet
from data.UnetDataset import ImgData
from torch.utils.data import Dataset, DataLoader,random_split

In [4]:
img_path=r""
mask_path=r""

In [5]:
trans1 = transforms.Compose([
    transforms.Resize([448,320]),
    transforms.ToTensor()
])

In [6]:
dataset=ImgData(img_path,mask_path,trans1)
train,val=random_split(dataset,[0.9,0.1])

In [7]:
train_loader = DataLoader(train,batch_size=8,shuffle=True)


In [8]:
model = Unet()
model=model.to('cuda')
Epoch=50
LR=1e-4
criterion=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=LR)


def get_IOU(SR,GT,threshold=0.5):
    SR = SR > threshold
    GT = GT == torch.max(GT)
    # 计算交集和并集
    intersection = (SR * GT).sum()
    union = SR.sum() + GT.sum() - intersection
    
    # 避免除以零
    iou = intersection / (union + 1e-6)
    
    return iou
    

In [9]:
save_path=r"F:\items\Seg\SegInOne\model_zoo\Unet.pth"
best_iou=0.0
for epoch in range(Epoch):
    
    model.train()
    for i,data in enumerate(train_loader):
        img,mask=data
        output=model(img.to('cuda'))
        loss=criterion(output,mask.to('cuda'))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss)
    
    model.eval()
    total_iou=0.0
    with torch.no_grad():
        for data in val:
            img,mask=data
            img=img.to('cuda')
            mask=mask.to('cuda')
            output=model(img.unsqueeze(0))
            iou=get_IOU(SR=output.squeeze(0),GT=mask)
            total_iou+=iou
        avg_iou=total_iou/len(val)
        if avg_iou>best_iou:
            best_iou=avg_iou
            print("new best model")
            torch.save(model.state_dict(),save_path)
        

tensor(0.8536, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.7173, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6585, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6175, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6313, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6281, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6216, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6095, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6118, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6014, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6061, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.5954, device='cuda:0',


KeyboardInterrupt: 

In [32]:
model = Unet()
model=model.to('cuda')
model.load_state_dict(torch.load(save_path))
model.eval()

test_img_path="./Unet/test_unet.jpg"
test_out_path="./Unet/output.jpg"
test_img=Image.open(test_img_path)
w,h=test_img.size
print(w,h)

trans2=transforms.Compose([
    transforms.ToPILImage()
])

test_input=trans1(test_img)

test_output=model(test_input.to('cuda').unsqueeze(0))
test_output=test_output.squeeze(0)
print(test_output)

test_output=transforms.functional.resize(
    test_output.unsqueeze(0),  # 添加通道维度
    size=(h, w), 
    interpolation=transforms.InterpolationMode.NEAREST
).squeeze(0)  # 移除通道维度

# 将logits转换为概率
prob_output = torch.sigmoid(test_output)


# 创建二值掩码（0和1）
binary_mask = (prob_output > 0.5).float()  # 转换为浮点数张量

print(binary_mask.max())
test_output_img=trans2(binary_mask)
test_output_img.save(test_out_path)

1918 1280
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor(1., device='cuda:0')


In [34]:
import torchvision
resnet=torchvision.models.resnet18(weights=None)

In [37]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  